In [3]:
pip install pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

In [11]:
spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro") \
    .getOrCreate()


In [19]:
df_nomes = spark.read.csv("nomes_aleatorios.txt", header=False, inferSchema=False)

In [22]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

In [23]:
df_nomes.show(5)

+----------------+
|           Nomes|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [24]:
df_nomes.printSchema()

root
 |-- Nomes: string (nullable = true)



In [25]:
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [27]:
from pyspark.sql import functions as F

escolaridades = ["Fundamental", "Médio", "Superior"]

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    F.when(F.round(F.rand() * 2) == 0, F.lit(escolaridades[0]))
     .when(F.round(F.rand() * 2) == 1, F.lit(escolaridades[1]))
     .otherwise(F.lit(escolaridades[2]))
)

In [28]:
df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|       Médio|
|    Jamie Russell| Fundamental|
|   Edward Kistler|       Médio|
|    Sheila Maurer| Fundamental|
| Donald Golightly|    Superior|
|       David Gray|    Superior|
|      Joy Bennett|    Superior|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|    Superior|
|    Brian Farrell|       Médio|
+-----------------+------------+
only showing top 10 rows



In [82]:
# Lista de países sul-americanos
paises_sul_americanos = [
    'Argentina', 'Bolívia', 'Brasil', 'Chile', 'Colômbia', 'Equador', 'Guiana', 'França (Guiana Francesa)', 
    'Paraguai', 'Peru', 'Suriname', 'Uruguai', 'Venezuela'
]

# Converter a lista de países para uma string compatível com a função `array` do Spark
paises_array = ", ".join([f'"{pais}"' for pais in paises_sul_americanos])

# Criar a nova coluna "País" com valores aleatórios da lista
df_nomes = df_nomes.withColumn(
    "País",
    F.expr(f"array({paises_array})[cast(rand() * {len(paises_sul_americanos)} as int) + 1]")
)


In [83]:
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     País|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|       Médio| Colômbia|         1965|
|    Jamie Russell| Fundamental| Paraguai|         1986|
|   Edward Kistler|       Médio|Venezuela|         1989|
|    Sheila Maurer| Fundamental| Paraguai|         1996|
| Donald Golightly|    Superior|  Bolívia|         1962|
|       David Gray|    Superior|   Brasil|         1948|
|      Joy Bennett|    Superior|     Peru|         1963|
|      Paul Kriese| Fundamental| Colômbia|         1994|
|Berniece Ornellas|    Superior|Venezuela|         1997|
|    Brian Farrell|       Médio|Venezuela|         1993|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [84]:
df_nomes = df_nomes.withColumn(
    "AnoNascimento",
    F.round(F.rand() * (2010 - 1945) + 1945).cast("int")
)


In [85]:
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     País|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|       Médio| Colômbia|         1948|
|    Jamie Russell| Fundamental| Paraguai|         1982|
|   Edward Kistler|       Médio|Venezuela|         2002|
|    Sheila Maurer| Fundamental| Paraguai|         1983|
| Donald Golightly|    Superior|  Bolívia|         1950|
|       David Gray|    Superior|   Brasil|         1997|
|      Joy Bennett|    Superior|     Peru|         1986|
|      Paul Kriese| Fundamental| Colômbia|         1953|
|Berniece Ornellas|    Superior|Venezuela|         1955|
|    Brian Farrell|       Médio|Venezuela|         2003|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [86]:
df_select = df_nomes.filter(F.col("AnoNascimento") >= 2001)

In [87]:
df_select.show(10)

+----------------+------------+---------+-------------+
|           Nomes|Escolaridade|     País|AnoNascimento|
+----------------+------------+---------+-------------+
|  Edward Kistler|       Médio|Venezuela|         2002|
|   Brian Farrell|       Médio|Venezuela|         2003|
|    David Medina|       Médio|     NULL|         2009|
|     Albert Leef|       Médio|   Brasil|         2006|
|         Lois Ly|    Superior|   Guiana|         2010|
|Wallace Mitchell| Fundamental| Paraguai|         2010|
| Jerry Chynoweth|       Médio|  Uruguai|         2007|
|   Lynne Dustman| Fundamental|  Equador|         2005|
| Katherine Moore|    Superior|    Chile|         2008|
|     Rita Walter| Fundamental|  Bolívia|         2002|
+----------------+------------+---------+-------------+
only showing top 10 rows



In [88]:
df_select.select("Nomes").show(10)

+----------------+
|           Nomes|
+----------------+
|  Edward Kistler|
|   Brian Farrell|
|    David Medina|
|     Albert Leef|
|         Lois Ly|
|Wallace Mitchell|
| Jerry Chynoweth|
|   Lynne Dustman|
| Katherine Moore|
|     Rita Walter|
+----------------+
only showing top 10 rows



In [89]:
df_nomes.createOrReplaceTempView("pessoas")
spark.sql("select Nomes from pessoas").show(10)


+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [90]:
df_nomes.filter((F.col("AnoNascimento") >= 1980) & (F.col("AnoNascimento") <= 1994)).count()

2306498

In [91]:
result = spark.sql("select count(Nomes) from pessoas where AnoNascimento between 1980 and 1994")
result.show()

+------------+
|count(Nomes)|
+------------+
|     2306498|
+------------+



In [92]:
df_nomes = df_nomes.withColumn(
    "Geração",
    F.when((F.col("AnoNascimento") >= 1944) & (F.col("AnoNascimento") <= 1964), "Baby Boomers")
    .when((F.col("AnoNascimento") >= 1965) & (F.col("AnoNascimento") <= 1979), "Geração X")
    .when((F.col("AnoNascimento") >= 1980) & (F.col("AnoNascimento") <= 1994), "Millennials")
    .when((F.col("AnoNascimento") >= 1995) & (F.col("AnoNascimento") <= 2015), "Geração Z")
    .otherwise("Outra")
)

df_geracoes = df_nomes.groupBy("País", "Geração").agg(F.count("*").alias("Quantidade"))
df_geracoes = df_geracoes.orderBy("País", "Geração", "Quantidade")
df_nomes = df_nomes.drop("Geração")

In [93]:
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     País|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|       Médio| Colômbia|         1948|
|    Jamie Russell| Fundamental| Paraguai|         1982|
|   Edward Kistler|       Médio|Venezuela|         2002|
|    Sheila Maurer| Fundamental| Paraguai|         1983|
| Donald Golightly|    Superior|  Bolívia|         1950|
|       David Gray|    Superior|   Brasil|         1997|
|      Joy Bennett|    Superior|     Peru|         1986|
|      Paul Kriese| Fundamental| Colômbia|         1953|
|Berniece Ornellas|    Superior|Venezuela|         1955|
|    Brian Farrell|       Médio|Venezuela|         2003|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [94]:
df_geracoes.show(100)

+--------------------+------------+----------+
|                País|     Geração|Quantidade|
+--------------------+------------+----------+
|                NULL|Baby Boomers|    230698|
|                NULL|   Geração X|    177677|
|                NULL|   Geração Z|    183943|
|                NULL| Millennials|    177312|
|             Bolívia|Baby Boomers|    230795|
|             Bolívia|   Geração X|    177182|
|             Bolívia|   Geração Z|    183504|
|             Bolívia| Millennials|    177763|
|              Brasil|Baby Boomers|    230461|
|              Brasil|   Geração X|    177604|
|              Brasil|   Geração Z|    183423|
|              Brasil| Millennials|    177599|
|               Chile|Baby Boomers|    230540|
|               Chile|   Geração X|    177915|
|               Chile|   Geração Z|    183452|
|               Chile| Millennials|    176838|
|            Colômbia|Baby Boomers|    231660|
|            Colômbia|   Geração X|    176939|
|            